In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tcn import TCN, tcn_full_summary
from optuna.samplers import TPESampler,CmaEsSampler
import optuna
batch_size, time_steps, input_dim = 20, 5, 9
dataset=pd.read_csv('SolarPrediction_aligned_Sun.csv',engine='python',nrows=576*15)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
dataset.head(5)
dataset=dataset.values


In [38]:

dataset=dataset.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar_dim=dataset[:,1]
dataset=scalar1.fit_transform(dataset)

scalar_dim=scalar_dim.reshape(len(dataset),1)
print(scalar_dim)
scalar_dim=scalar2.fit_transform(scalar_dim)
print(scalar_dim)


[[  2.58]
 [  2.83]
 [  2.16]
 ...
 [344.9 ]
 [365.66]
 [355.57]]
[[0.00089369]
 [0.00104993]
 [0.00063121]
 ...
 [0.21482898]
 [0.2278031 ]
 [0.22149728]]


In [39]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]


In [40]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,1])
    return np.array(dataX),np.array(dataY)

    

In [41]:
look_back=5
trainX,trainY=create_dataset(train,look_back)
testX,testY=create_dataset(test,look_back)

In [42]:
def objective(trial):
    tcn_layer = TCN(
                    nb_filters=trial.suggest_int("nb_filters", 1, 128),
                    kernel_size=trial.suggest_int("kernel_size",5,10),
                    dropout_rate=trial.suggest_float("dropout_rate",0,0.5),
                    #activation=trial.suggest_categorical("activation",['relu','tanh','linear']),
                    padding='causal',
                    #nb_stacks=trial.suggest_int("nb_stacks", 1, 3),
                    input_shape=(time_steps, input_dim))
    print('Receptive field size =', tcn_layer.receptive_field)

    model = Sequential([tcn_layer,Dense(1)])
    model.compile(optimizer='adam', loss='mse')
    tcn_full_summary(model, expand_residual_blocks=False)
    model.fit(trainX, trainY, epochs=1, validation_split=0.2)
    # 進行預測
    y_pred = model.predict(testX)

    # 計算均方誤差
    mse = mean_squared_error(testY, y_pred)

    return mse

In [43]:
#search={"nb_filters": [16,128], "kernel_size":[5,10],"dropout_rate":[0.1,0.5]}
#sampler = optuna.samplers.GridSampler(search)
study = optuna.create_study(direction="minimize",sampler=TPESampler())
study.optimize(objective, n_trials=30)

# 取得最佳超參數
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2024-03-04 03:40:36,548] A new study created in memory with name: no-name-2bda661b-8079-499a-960a-6c1ef710590f


Receptive field size = 1009
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.
89/89 [==============================] - 2s 22ms/step


[I 2024-03-04 03:40:52,632] Trial 0 finished with value: 0.012151566334068775 and parameters: {'nb_filters': 108, 'kernel_size': 9, 'dropout_rate': 0.2520672394350867}. Best is trial 0 with value: 0.012151566334068775.


Receptive field size = 883
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.
89/89 [==============================] - 2s 18ms/step


[I 2024-03-04 03:41:07,043] Trial 1 finished with value: 0.006552626378834248 and parameters: {'nb_filters': 108, 'kernel_size': 8, 'dropout_rate': 0.06430619176014324}. Best is trial 1 with value: 0.006552626378834248.


...

Receptive field size = 1135
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.
89/89 [==============================] - 3s 31ms/step


[I 2024-03-04 03:42:01,945] Trial 5 finished with value: 0.005103816743940115 and parameters: {'nb_filters': 108, 'kernel_size': 10, 'dropout_rate': 0.06283420076202606}. Best is trial 5 with value: 0.005103816743940115.


Receptive field size = 883
Use tensorboard instead. Example in keras-tcn/tasks/tcn_tensorboard.py.


[W 2024-03-04 03:42:07,006] Trial 6 failed with parameters: {'nb_filters': 21, 'kernel_size': 8, 'dropout_rate': 0.3733664493569508} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\wang8\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\wang8\AppData\Local\Temp\ipykernel_3156\3371034517.py", line 15, in objective
    model.fit(trainX, trainY, epochs=1, validation_split=0.2)
  File "C:\Users\wang8\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\wang8\anaconda3\lib\site-packages\keras\src\engine\training.py", line 1783, in fit
    tmp_logs = self.train_function(iterator)
  File "C:\Users\wang8\anaconda3\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "C:\Users\wang8\anaconda3\lib\site-

KeyboardInterrupt: 